<a href="https://colab.research.google.com/github/stavco9/datastreaming-final-project/blob/main/Data_Analysis_DeezyMatch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install DeezyMatch
!pip install thefuzz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7

In [2]:
import os
import tqdm
import time
from DeezyMatch import candidate_ranker, candidate_ranker_init
from thefuzz import process, fuzz

/usr/local/lib/python3.10/dist-packages/DeezyMatch/rnn_networks.py:30: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/IDC_MSc/Year1/DataStreamingAlgorithms/DeezyMatchDataset"

Mounted at /content/drive
/content/drive/My Drive/IDC_MSc/Year1/DataStreamingAlgorithms/DeezyMatchDataset


In [4]:
words_set = []
for i in range(10):
  candidates_file = f"./candidates/dataset_candidates_{i}.txt"
  with open(candidates_file, 'r') as f:
    words_list = f.read().split('\n')
    words_set.append({"words_list": words_list})

In [5]:
some_sentance_with_error = "I wanna appel pleases or a bannana with a paer and a lemone, \
or a wartermelon if possibe. I hope i don't have an i \
don't have an errof in Londaon and in Jerusealm"

In [9]:
for index, words_list in enumerate(words_set):
    time_start_naive = time.time()
    words_set[index]["naive_matches"] = {}
    words_set[index]["naive_matches"]["origin_words"] = []
    words_set[index]["naive_matches"]["matched_words"] = []
    words_set[index]["naive_matches"]["match_scores"] = []
    for word in some_sentance_with_error.split():
      words_set[index]["naive_matches"]["origin_words"].append(word)
      word_match_results = [x for x in process.extract(word, words_list["words_list"], scorer=fuzz.ratio)]
      words_set[index]["naive_matches"]["matched_words"].append(word_match_results[0][0] if len(word_match_results) > 0 else "None")
      words_set[index]["naive_matches"]["match_scores"].append(int(word_match_results[0][1]) / 100 if len(word_match_results) > 0 else 0)
    time_end_naive = time.time()
    words_set[index]["time_naive_sec"] = (time_end_naive - time_start_naive)

In [10]:
for index, words_list in enumerate(words_set):
   print(f"Total time of dataset {index + 1} is {words_list['time_naive_sec']}s")
   print(f"Origin words for dataset {index + 1} are {words_list['naive_matches']['origin_words']}")
   print(f"Matches words for dataset {index + 1} are {words_list['naive_matches']['matched_words']}")
   print(f"Matches scores for dataset {index + 1} are {words_list['naive_matches']['match_scores']}")

Total time of dataset 1 is 6.014449119567871s
Origin words for dataset 1 are ['I', 'wanna', 'appel', 'pleases', 'or', 'a', 'bannana', 'with', 'a', 'paer', 'and', 'a', 'lemone,', 'or', 'a', 'wartermelon', 'if', 'possibe.', 'I', 'hope', 'i', "don't", 'have', 'an', 'i', "don't", 'have', 'an', 'errof', 'in', 'Londaon', 'and', 'in', 'Jerusealm']
Matches words for dataset 1 are ['I', 'Wanna', 'appeal', 'Please', 'or', 'a', 'bananas', 'with', 'a', 'paper', 'and', 'a', 'Lemon', 'or', 'a', 'Waterloo', 'if', 'possible?!', 'I', 'Hope.', 'I', "don't", 'have', 'an', 'I', "don't", 'have', 'an', 'error', 'in', 'London', 'and', 'in', 'Jerusalem']
Matches scores for dataset 1 are [1.0, 1.0, 0.91, 0.92, 1.0, 1.0, 0.86, 1.0, 1.0, 0.89, 1.0, 1.0, 0.91, 1.0, 1.0, 0.74, 1.0, 0.93, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8, 1.0, 0.92, 1.0, 1.0, 0.89]
Total time of dataset 2 is 5.98909068107605s
Origin words for dataset 2 are ['I', 'wanna', 'appel', 'pleases', 'or', 'a', 'bannana', 'with', 'a', 'p

In [15]:
for index in [0, 1, 3, 4, 5, 6]:
  # Ranking on-the-fly
  # find candidates from candidate_scenario
  # for queries specified by the `query` argument
  time_start_deezy = time.time()
  ranker = \
      candidate_ranker_init(candidate_scenario=f"./combined/candidates_test_{i}",
                      query=[x.replace("-", " ") for x in some_sentance_with_error.split()],
                      ranking_metric="faiss",
                      selection_threshold=10.,
                      num_candidates=1,
                      search_size=100,
                      output_path=f"ranker_results/test_candidates_deezymatch_on_the_fly_{i}",
                      pretrained_model_path="./models/wikigaz_en_model/wikigaz_en_model.model",
                      pretrained_vocab_path="./models/wikigaz_en_model/wikigaz_en_model.vocab",
                      number_test_rows=len(some_sentance_with_error))

  ranker.rank()
  words_set[index]["deezy_match_df"] = ranker.output
  time_end_deezy = time.time()
  words_set[index]["time_deezy_sec"] = (time_end_deezy - time_start_deezy)

2024-03-23 21:00:44 a7691face663 [INFO] read input file: ./combined/candidates_test_6/train_conf.yaml
2024-03-23 21:00:44 a7691face663 [INFO] pytorch will use: cuda
2024-03-23 21:00:45 a7691face663 [INFO] use a dataframe in test_tokenize.
2024-03-23 21:00:45 a7691face663 [INFO] number of labels, True: 0 and False: 34
2024-03-23 21:00:45 a7691face663 [INFO] skipping 0 lines


2024-03-23 21:00:45 a7691face663 [INFO] save test-data-class: tmp_f9d8d94b-a766-425b-ab03-0c092618eba8/query/dataframe.df


  0%|          | 0/1 [00:00<?, ?it/s]



-- Combine vectors
Reading vectors from tmp_f9d8d94b-a766-425b-ab03-0c092618eba8/query/embeddings/rnn_fwd*
0000000 tmp_f9d8d94b-a766-425b-ab03-0c092618eba8/query/embeddings/rnn_fwd_0


-- Combine IDs

0000000 tmp_f9d8d94b-a766-425b-ab03-0c092618eba8/query/embeddings/rnn_indxs_0


-- Combine vectors
Reading vectors from tmp_f9d8d94b-a766-425b-ab03-0c092618eba8/query/embeddings/rnn_bwd*
0000000 tmp_f9d8d94b-a766-425b-ab03-0c092618eba8/query/embeddings/rnn_bwd_0


-- Combine IDs

0000000 tmp_f9d8d94b-a766-425b-ab03-0c092618eba8/query/embeddings/rnn_indxs_0
--- 1841.8998532295227 seconds ---
Is faiss_id_candis already trained? True
=========== Start the search for 0 I
ID: 1/34 -- Number of found candidates so far: 3, searched: 100
=========== Start the search for 1 wanna
ID: 2/34 -- Number of found candidates so far: 4, searched: 100
=========== Start the search for 2 appel
ID: 3/34 -- Number of found candidates so far: 14, searched: 100
=========== Start the search for 3 pleases
ID: 4/3

2024-03-23 21:00:48 a7691face663 [INFO] save test-data-class: tmp_42a74843-7b00-4496-94bc-c00e10d5edf8/query/dataframe.df


...

  0%|          | 0/1 [00:00<?, ?it/s]



-- Combine vectors
Reading vectors from tmp_e4b6ffb7-4e8c-46c7-87e1-2c4d32b32865/query/embeddings/rnn_fwd*
0000000 tmp_e4b6ffb7-4e8c-46c7-87e1-2c4d32b32865/query/embeddings/rnn_fwd_0


-- Combine IDs

0000000 tmp_e4b6ffb7-4e8c-46c7-87e1-2c4d32b32865/query/embeddings/rnn_indxs_0


-- Combine vectors
Reading vectors from tmp_e4b6ffb7-4e8c-46c7-87e1-2c4d32b32865/query/embeddings/rnn_bwd*
0000000 tmp_e4b6ffb7-4e8c-46c7-87e1-2c4d32b32865/query/embeddings/rnn_bwd_0


-- Combine IDs

0000000 tmp_e4b6ffb7-4e8c-46c7-87e1-2c4d32b32865/query/embeddings/rnn_indxs_0
--- 1852.5213656425476 seconds ---
Is faiss_id_candis already trained? True
=========== Start the search for 0 I
ID: 1/34 -- Number of found candidates so far: 3, searched: 100
=========== Start the search for 1 wanna
ID: 2/34 -- Number of found candidates so far: 4, searched: 100
=========== Start the search for 2 appel
ID: 3/34 -- Number of found candidates so far: 14, searched: 100
=========== Start the search for 3 pleases
ID: 4/3

2024-03-23 21:00:58 a7691face663 [INFO] save test-data-class: tmp_34874daa-1bd2-47e0-ba73-c347607c2868/query/dataframe.df


  0%|          | 0/1 [00:00<?, ?it/s]



-- Combine vectors
Reading vectors from tmp_34874daa-1bd2-47e0-ba73-c347607c2868/query/embeddings/rnn_fwd*
0000000 tmp_34874daa-1bd2-47e0-ba73-c347607c2868/query/embeddings/rnn_fwd_0


-- Combine IDs

0000000 tmp_34874daa-1bd2-47e0-ba73-c347607c2868/query/embeddings/rnn_indxs_0


-- Combine vectors
Reading vectors from tmp_34874daa-1bd2-47e0-ba73-c347607c2868/query/embeddings/rnn_bwd*
0000000 tmp_34874daa-1bd2-47e0-ba73-c347607c2868/query/embeddings/rnn_bwd_0


-- Combine IDs

0000000 tmp_34874daa-1bd2-47e0-ba73-c347607c2868/query/embeddings/rnn_indxs_0
--- 1854.577793121338 seconds ---
Is faiss_id_candis already trained? True
=========== Start the search for 0 I
ID: 1/34 -- Number of found candidates so far: 3, searched: 100
=========== Start the search for 1 wanna
ID: 2/34 -- Number of found candidates so far: 4, searched: 100
=========== Start the search for 2 appel
ID: 3/34 -- Number of found candidates so far: 14, searched: 100
=========== Start the search for 3 pleases
ID: 4/34

In [26]:
for index in [0, 1, 3, 4, 5, 6]:
    deezy_match_df = words_set[index]["deezy_match_df"]
    words_set[index]["deezy_matches"] = {}
    words_set[index]["deezy_matches"]["origin_words"] = []
    words_set[index]["deezy_matches"]["matched_words"] = []
    words_set[index]["deezy_matches"]["match_distance"] = []
    for word in some_sentance_with_error.split():
      word_df_row = deezy_match_df.loc[deezy_match_df['query'] == word].iloc[0]
      words_set[index]["deezy_matches"]["origin_words"].append(word)
      words_set[index]["deezy_matches"]["matched_words"].append(list(word_df_row['faiss_distance'].keys())[0] if bool(word_df_row['faiss_distance']) else None)
      words_set[index]["deezy_matches"]["match_distance"].append(list(word_df_row['faiss_distance'].values())[0] if bool(word_df_row['faiss_distance']) else -1)

In [28]:
for index in [0, 1, 3, 4, 5, 6]:
   print(f"Total time of dataset {index + 1} is {words_set[index]['time_deezy_sec']}s")
   print(f"Origin words for dataset {index + 1} are {words_set[index]['deezy_matches']['origin_words']}")
   print(f"Matches words for dataset {index + 1} are {words_set[index]['deezy_matches']['matched_words']}")
   print(f"Matches scores for dataset {index + 1} are {words_set[index]['deezy_matches']['match_distance']}")

Total time of dataset 1 is 3.0817720890045166s
Origin words for dataset 1 are ['I', 'wanna', 'appel', 'pleases', 'or', 'a', 'bannana', 'with', 'a', 'paer', 'and', 'a', 'lemone,', 'or', 'a', 'wartermelon', 'if', 'possibe.', 'I', 'hope', 'i', "don't", 'have', 'an', 'i', "don't", 'have', 'an', 'errof', 'in', 'Londaon', 'and', 'in', 'Jerusealm']
Matches words for dataset 1 are ['I', 'Hanna', 'appealed', 'peeresses', 'OR', 'a', 'Banana', 'with', 'a', 'Parker', 'and', 'a', 'Leone.', 'OR', 'a', 'warned', 'if', 'possible.', 'I', 'Hope', 'I', "don't", 'have', 'an', 'I', "don't", 'have', 'an', 'error', 'in', 'London', 'and', 'in', 'Jerusalem']
Matches scores for dataset 1 are [0.0, 7.2827, 6.1974, 5.3401, 0.0, 0.0, 3.6072, 0.0, 0.0, 3.4475, 0.0, 0.0, 6.0364, 0.0, 0.0, 7.5992, 0.0, 4.5969, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.2706, 0.0, 3.8871, 0.0, 0.0, 5.5064]
Total time of dataset 2 is 2.030217409133911s
Origin words for dataset 2 are ['I', 'wanna', 'appel', 'pleases', 'or', 'a'